# 추천 시스템

## 추천 시스템의 중요성
- 아마존 등과 같은 전자상거래 업체부터 넷플릭스, 유튜브 등 콘텐츠 포털까지 추천시스템을 통해 **사용자의 취향을 이해하고 맞춤 상품과 콘텐츠를 제공**하고 있다.
- 조금이라도 오랫동안 서비스에 고객을 오래 머무르게 하기 위해 전력을 기울이고 있다.

## 추천 엔진의 필요성
- 상품 콘텐츠 서비스를 제공하는 측에서는 너무나 많은 상품이 존재
- 이에 따라 사용자는 어떤 상품을 골라야 할지 선택의 압박이 생긴다.
- 추천엔진은 사용자가 무엇을 원하는지 빠르게 찾아내어 사용자의 온라인 쇼핑 및 콘텐츠 선택에 있어 즐거움이 생기게 된다.

## 추천 시스템 방식
1. 콘텐츠 기반 필터링(Content Based Filitering)
  - 상품의 속성에 개인의 취향을 반영
  - 사용자에게 콘텐츠가 가지고 있는 여러 요소가 사용자에게 어울리는 지 판별해서 제공
  - ex. 어떤 사용자가 특정 장르를 조항한다면 특정 장르 영화 중심으로 추천
2. 협업 필터링(Collaborative Filtering)
  - 나와 비슷한 사람이 선택한 상품을 선택한다는 개념
  - ex. A와 B의 쇼핑목록이 비슷할 때, A가 샀지만 B는 사지않은 물건을 B에게 추천

- 추천 시스템은 두 방식 중 한 가지를 선택하거나 이들을 결합하여 하이브리드 방식으로 사용한다.
- 요즘에는 딥러닝으로 추천 시스템을 만들기도 한다.

## 콘텐츠 기반 필터링(Contents Based Filtering)
- 실수 형식의 정형화된 데이터 뿐만 아니라 텍스트 기반으로 한 비정형화 데이터도 사용할 수 있다.
- 데이터의 유사도를 비교하여 추천한다.
- 비정형화된 데이터는 Feature Vectorization(피처 벡터화)를 하여 코사인 유사도(Cosine Similarity)를 구한다.

## 콘텐츠 기반 필터링 실습
- 영화 평점 사이트 IMDB 데이터 세트를 캐글에서 다운로드 받아 가져온다.

### 콘텐츠 기반 필터링 프로세스
1. 콘텐츠에 대한 여러 텍스트 정보들을 피처 벡터화
2. 코사인 유사도로 콘텐츠별 유사도 계산
3. 콘텐츠 별로 가중 평점을 계산
4. 유사도가 높은 콘텐츠 중에 평점이 좋은 콘텐츠 순으로 추천

### 1. 데이터 가져오기

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pandas as pd

# 표시되는 컬럼의 가로 길이 키우기
pd.set_option("max_colwidth", 200)

DATA_PATH = "/content/drive/MyDrive/MLP-33-ML-DL/ML/실습_data/tmdb_5000_movies.csv"

movies = pd.read_csv(DATA_PATH)
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 878, ""name"": ""Science Fiction""}]",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""space war""}, {""id"": 3388, ""name"": ""space colony""}, {""id"": 3679, ""name"": ""society""}, {""id"": 3801, ""name...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporation"", ""id"": 306}, {""name"": ""Dune Entertainment"", ""id"": 444}, {""name"": ""Lightstorm Entertainment"", ""id""...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}]",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic island""}, {""id"": 1319, ""name"": ""east india trading company""}, {""id"": 2038, ""name"": ""love of one's life...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""name"": ""Jerry Bruckheimer Films"", ""id"": 130}, {""name"": ""Second Mate Productions"", ""id"": 19936}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret agent""}, {""id"": 9663, ""name"": ""sequel""}, {""id"": 14555, ""name"": ""mi6""}, {""id"": 156095, ""name"": ""brit...",en,Spectre,"A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit ...",107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""name"": ""Danjaq"", ""id"": 10761}, {""name"": ""B24"", ""id"": 69434}]","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}, {""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""}, {""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}, {""iso_639_1"": ""it"", ""name"": ""Italiano""}, {""iso_639_1"": ""de"", ""na...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name"": ""Drama""}, {""id"": 53, ""name"": ""Thriller""}]",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853, ""name"": ""crime fighter""}, {""id"": 949, ""name"": ""terrorist""}, {""id"": 1308, "

### 필요한 Feature만 선택
- 실습에서는 genres, keywords 중심으로 콘텐츠 기반 필터링을 수행하겠다.

In [5]:
movies_df = movies[["id", "title", "genres", "vote_average", "vote_count", "popularity", "keywords", "overview"]]
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


In [6]:
movies_df[["genres", "keywords"]].head()

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 878, ""name"": ""Science Fiction""}]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""space war""}, {""id"": 3388, ""name"": ""space colony""}, {""id"": 3679, ""name"": ""society""}, {""id"": 3801, ""name..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic island""}, {""id"": 1319, ""name"": ""east india trading company""}, {""id"": 2038, ""name"": ""love of one's life..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret agent""}, {""id"": 9663, ""name"": ""sequel""}, {""id"": 14555, ""name"": ""mi6""}, {""id"": 156095, ""name"": ""brit..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name"": ""Drama""}, {""id"": 53, ""name"": ""Thriller""}]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853, ""name"": ""crime fighter""}, {""id"": 949, ""name"": ""terrorist""}, {""id"": 1308, ""name"": ""secret identity""}, {""id"": 1437, ""name"": ""burglar""}, {""id"": 3051, ""n..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 878, ""name"": ""Science Fiction""}]","[{""id"": 818, ""name"": ""based on novel""}, {""id"": 839, ""name"": ""mars""}, {""id"": 1456, ""name"": ""medallion""}, {""id"": 3801, ""name"": ""space travel""}, {""id"": 7376, ""name"": ""princess""}, {""id"": 9951, ""name"":..."


- genres, keyword의 값이 비정형 데이터인 json array 형태인 것을 확인
  - json : 자바스크립트 객체 표현 방식 중 하나로 구조화된 데이터를 표현하기 위한 문자 기반의 표준 포맷
  - `{"id": 28, "name": "Action"}`
  - 파이썬에서 dict 형식과 유사하다.
  - json이 리스트 형태에 담겨있는 것이 json array
- genres, keyword의 값이 json 형식의 문자열이지만, `movies_df.info()`에서 확인해보면 `test(object)` 형식으로 되어있기 때문에, 해당값에서 실제 장르만 추출해내기 위한 가공작업이 따로 필요하다.
  

In [7]:
import json

# json.loads : json 형식의 문자열을 json 객체 타입으로 바꿔 준다.
# apply() 함수로 genres, keywords 컬럼 전체에 json.loads 함수를 적용한다.
movies_df['genres'] = movies_df['genres'].apply(json.loads)
movies_df['keywords'] = movies_df['keywords'].apply(json.loads)

movies_df.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 878, 'name': 'Science Fiction'}]",7.2,11800,150.437577,"[{'id': 1463, 'name': 'culture clash'}, {'id': 2964, 'name': 'future'}, {'id': 3386, 'name': 'space war'}, {'id': 3388, 'name': 'space colony'}, {'id': 3679, 'name': 'society'}, {'id': 3801, 'name...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization."
1,285,Pirates of the Caribbean: At World's End,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 28, 'name': 'Action'}]",6.9,4500,139.082615,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'name': 'drug abuse'}, {'id': 911, 'name': 'exotic island'}, {'id': 1319, 'name': 'east india trading company'}, {'id': 2038, 'name': 'love of one's life...","Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems."
2,206647,Spectre,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 80, 'name': 'Crime'}]",6.3,4466,107.376788,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name': 'based on novel'}, {'id': 4289, 'name': 'secret agent'}, {'id': 9663, 'name': 'sequel'}, {'id': 14555, 'name': 'mi6'}, {'id': 156095, 'name': 'brit...","A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit ..."
3,49026,The Dark Knight Rises,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 18, 'name': 'Drama'}, {'id': 53, 'name': 'Thriller'}]",7.6,9106,112.312950,"[{'id': 849, 'name': 'dc comics'}, {'id': 853, 'name': 'crime fighter'}, {'id': 949, 'name': 'terrorist'}, {'id': 1308, 'name': 'secret identity'}, {'id': 1437, 'name': 'burglar'}, {'id': 3051, 'n...","Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police..."
4,49529,John Carter,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 878, 'name': 'Science Fiction'}]",6.1,2124,43.926995,"[{'id': 818, 'name': 'based on novel'}, {'id': 839, 'name': 'mars'}, {'id': 1456, 'name': 'medallion'}, {'id': 3801, 'name': 'space travel'}, {'id': 7376, 'name': 'princess'}, {'id': 9951, 'name':...","John Carter is a war-weary, former military captain who's inexplicably transported to the mysterious and exotic planet of Barsoom (Mars) and reluctantly becomes embroiled in an epic conflict. It's..."


In [8]:
## json 객체에서 특정 key의 value 값 반환하는 방법
sample = [{'id': 28, 'name': 'Action'},
          {'id': 12, 'name': 'Adventure'},
          {'id': 14, 'name': 'Fantasy'},
          {'id': 878, 'name': 'Science Fiction'}]

[data['name'] for data in sample]

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

- 위 방법을 적용해 genres와 keywords의 `name` key의 value만 반환해 기존 컬럼의 값들을 대체하겠다.

In [9]:
movies_df['genres'] = movies_df['genres'].apply(lambda datas : [data['name'] for data in datas ])
movies_df['keywords'] = movies_df['keywords'].apply(lambda datas : [data['name'] for data in datas ])

movies_df[["genres", "keywords"]].head()

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations..."
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india trading company, love of one's life, traitor, shipwreck, strong woman, ship, alliance, calypso, afterlife, fighter, pirate, swashbuckler, aftercredits..."
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi6, british secret service, united kingdom]"
3,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret identity, burglar, hostage drama, time bomb, gotham city, vigilante, cover-up, superhero, villainess, tragic hero, terrorism, destruction, catwoman, ca..."
4,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel, princess, alien, steampunk, martian, escape, edgar rice burroughs, alien race, superhuman strength, mars civilization, sword and planet, 19th centur..."


### Feature Vectorization
```python
[Action, Adventure, Fantasy, Science Fiction]
```
- 비정형 데이터의 벡터화
- genres에 대한 특정 장르 값이 있으면 1로 count하는 가장 간단한 방식으로 진행

In [10]:
## 장르 콘텐츠 필터링을 활용한 영화 추천 구현을 위해 장르 문자열을 Count 벡터화 수행

from sklearn.feature_extraction.text import CountVectorizer

# 리스트 형태는 잘 인식하지 못하기 못하기 때문에 모든 요소가 담긴 긴 문자열 문장으로 만들어 준다.
# CountVectorizer를 적용하기 위해 공백 문자로 word 단위로 구분되는 문자열로 변환 - join() 활용
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : ' '.join(x))
movies_df[['genres', 'genres_literal']].head()

,genres,genres_literal
0,"[Action, Adventure, Fantasy, Science Fiction]",Action Adventure Fantasy Science Fiction
1,"[Adventure, Fantasy, Action]",Adventure Fantasy Action
2,"[Action, Adventure, Crime]",Action Adventure Crime
3,"[Action, Crime, Drama, Thriller]",Action Crime Drama Thriller
4,"[Action, Adventure, Science Fiction]",Action Adventure Science Fiction


In [11]:
count_vect = CountVectorizer(min_df=0.0, ngram_range=(1,2))
genre_matrix = count_vect.fit_transform(movies_df['genres_literal'])
genre_matrix

<4803x276 sparse matrix of type '<class 'numpy.int64'>'
	with 20631 stored elements in Compressed Sparse Row format>

- row(영화개수) : 4803개
- column(영화 장르의 개수) : 276개

In [12]:
genre_matrix.toarray()

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

- 각 영화별로 장르에 포함된 개수를 count해 각각 1, 0이 담긴 배열 확인

### 코사인 유사도(Cosine Similarity)
- `genre_matrix` 행렬을 이용해 장르에 따른 영화 별 코사인 유사도 추출
  - 행렬의 0번째 인덱스값 `[1, 1, 0, ..., 0, 0, 0]` : count 벡터를 내적을 통해 0번째 인덱스 영화에 대한 나머지 영화에 대한 코사인 유사도를 구할 수 있다.
  - 동일한 방식으로 0번째부터 마지막 인덱스 영화에 대한 count 벡터에 대한 코사인 유사도 구하기
  - 전치행렬을 이용해 코사인 유사도를 구할 수 있다.
  - `genre_matrix`를 행렬 $A$라고 볼 때, $A \in \mathbf{R}^{4803 \times 276}$
  - $AA^T \in \mathbf{R}^{4803 \times 4803}$, $AA^T$는 대칭행렬이므로 유사도를 구할 수 있다.

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

## 사이킷런에서는 기본적으로 행벡터끼리의 유사도를 구하게 된다.
# 두번째 인자로 넣은 행렬을 자동으로 전치해서 연산해주기 때문에 전치행렬로 넣지 않아도 된다.
genre_sim = cosine_similarity(genre_matrix, genre_matrix)

genre_sim.shape

(4803, 4803)

- $AA^T \in \mathbf{R}^{4803 \times 4803}$ 코사인 유사도값이 담긴 행렬이 정방행렬(대칭행렬) 형태가 된 것을 확인

In [14]:
# 코사인 유사도 확인
genre_sim[:7]

array([[1.        , 0.59628479, 0.4472136 , ..., 0.        , 0.        ,
        0.        ],
       [0.59628479, 1.        , 0.4       , ..., 0.        , 0.        ,
        0.        ],
       [0.4472136 , 0.4       , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.75592895, 0.3380617 , 0.50709255, ..., 0.        , 0.        ,
        0.        ],
       [0.59628479, 0.8       , 0.6       , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

- 주대각 방향에 있는 값들은 1인 대칭행렬인 것을 확인

In [15]:
genre_sim[0]

array([1.        , 0.59628479, 0.4472136 , ..., 0.        , 0.        ,
       0.        ])

- `genre_sim[0][0]`은 0번째 영화와 0번째 영화의 유사도로 같은 영화이기 때문에 유사도가 1
- `genre_sim[0][1]`은 0번째 영화와 1번째 영화의 유사도로 `0.59628479` 값이 나왔다.

### 각 영화마다 코사인 유사도가 가장 높은 영화를 구하기 위해 정렬
- 1 값을 가지는 주대각 방향의 요소들은 나중에 제외하는 조건을 걸어줄 예정
- 열방향 벡터들을 코사인 유사도가 높은 순으로 정렬
  - `argsort()` 활용


- 샘플 데이터로 `argsort()`를 어떻게 활용하는지 먼저 확인해보겠다.

In [16]:
## 샘플 데이터로 argsort() 확인
import numpy as np

sample =  np.array([5, 3, 1, 2, 4])

# sort()로 오름차순 정렬시 값이 작은 순서대로 반환 => array([1, 2, 3, 4, 5])
# argsort() 정렬시 값이 작은 순서대로 리스트 내의 인덱스를 반환
sorted_index = sample.argsort()

print(f'index : {sorted_index}')
sample[sorted_index]

index : [2 3 1 4 0]


array([1, 2, 3, 4, 5])

In [17]:
# 이차원 배열에 대한 argsort()
# 행단위로 정렬되는 것을 확인할 수 있다.

sample = np.array([[4, 6, 1],
                   [7, 2, 9],
                   [8, 5, 3]])

sample.argsort()

array([[2, 0, 1],
       [1, 0, 2],
       [2, 1, 0]])

- 이차원 배열에 `argsort()`를 적용시 각 행에 대한 오름차수능로 정렬된 인덱스가 나오는 것을 확인할 수 있다.

### 영화 장르에 대한 코사인 유사도 행렬에 `argsort()` 적용
- `genre_sim.argsort()` : 각 영화마다 유사도가 낮은 순으로 정렬되어 리스트 내의 해당 인덱스가 반환 됨
- 유사도가 높은 순(내림차순)으로 인덱스를 정렬하기 위해 `[:, ::-1]` 사용

In [18]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1] # 내림차순으로 정렬하기 위해 모든 행 선택 후 모든 열 reverse

genre_sim_sorted_ind[:1]

array([[   0, 3494,  813, ..., 3038, 3037, 2401]])

In [19]:
## 데이터 프레임에서 영화 제목을 받아, 그 영화와 가장 비슷한 top 10을 추출해주는 함수 정의
# sorted_index에는 argsort()로 인덱스가 정렬된 코사인 유사도 함수가 들어간다.
def find_sim_movie(df, sorted_index, title_name, top_n=10):

  # 1. title_name 데이터 찾기
  title_movie = df[df['title'] == title_name] # 특정 영화에 대한 장르, 키워드를 포함한 정보 받아 올 수 있음

  #2. 찾고자 하는 영화의 index 찾기
  # index를 알아야 유사도 행렬의 몇 번째 벡터를 찾아야 할지 알 수 있다.
  title_index = title_movie.index.values

  # 정렬의 유사도가 아닌, 유사도의 인덱스를 가지고 온다.
  similar_indexes = sorted_index[title_index, :top_n] # argsort()를 통해 내림차순으로 인덱스가 정렬된 상태. top_n 만큼 슬라이싱해준다.

  # 1차원 배열로 만들어주기
  similar_indexes = similar_indexes.reshape(-1)

  # 데이터프레임의 iloc으로 인덱스 선택해서 해당 영화와 가장 비슷한 top 10을 추출
  return df.iloc[similar_indexes]

In [20]:
## 함수 적용해 아바타 영화와 가장 비슷한 top10을 추출
similar_movies_df = find_sim_movie(movies_df, genre_sim_sorted_ind, "Avatar", 10)

# 특정 컬럼에 대해서만 확인
similar_movies_df[['title', 'vote_average', 'genres', 'keywords']]

,title,vote_average,genres,keywords
0,Avatar,7.2,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations..."
3494,Beastmaster 2: Through the Portal of Time,4.6,"[Action, Adventure, Fantasy, Science Fiction]","[based on novel, time travel, sequel, psychotronic, sword and sandal, beastmaster, warrior, time portal, gun fight, sword and sorcery]"
813,Superman,6.9,"[Action, Adventure, Fantasy, Science Fiction]","[saving the world, journalist, dc comics, crime fighter, nuclear missile, galaxy, superhero, based on comic book, criminal, sabotage, north pole, midwest, kryptonite, super powers, superhuman stre..."
870,Superman II,6.5,"[Action, Adventure, Fantasy, Science Fiction]","[saving the world, dc comics, sequel, superhero, based on comic book, loss of virginity, criminal, super powers, phantom zone, rocket fired grenade, crystal machine, superhuman strength, duringcre..."
46,X-Men: Days of Future Past,7.5,"[Action, Adventure, Fantasy, Science Fiction]","[1970s, mutant, time travel, marvel comic, based on comic book, superhuman, storm, beast, aftercreditsstinger, changing the past or future]"
14,Man of Steel,6.5,"[Action, Adventure, Fantasy, Science Fiction]","[saving the world, dc comics, superhero, based on comic book, superhuman, alien invasion, reboot, super powers, dc extended universe]"
1296,Superman III,5.3,"[Comedy, Action, Adventure, Fantasy, Science Fiction]","[saving the world, dc comics, super computer, identity crisis, loss of powers, sequel, superhero, based on comic book, hacking, super powers, superhuman strength]"
1652,Dragonball Evolution,2.9,"[Action, Adventure, Fantasy, Science Fiction, Thriller]","[karate, superhero, revenge, dragon, duringcreditsstinger]"
419,Jumper,5.9,"[Adventure, Fantasy, Science Fiction]","[adolescence, based on novel, loss of child, fight, chase, teleportation, supernatural powers, leap in time, enemy, motherly love]"
420,Hellboy II: The Golden Army,6.5,"[Adventure, Fantasy, Science Fiction]","[auction, northern ireland, resignation, superhero, rebellion, violence, spear, cut arm, split screen, superhero team, arm ripped off, super villain, remorse, self exile, vanishing figure, father ..."


- 현재는 단순하게 만들었기 때문에 아바타 영화와 가장 유사한 영화로 아바타가 나왔다.



In [21]:
# 함수 적용해 갓파더 영화와 가장 비슷한 top10을 추출
similar_movies_df = find_sim_movie(movies_df, genre_sim_sorted_ind, "The Godfather", 10)

# 특정 컬럼에 대해서만 확인
similar_movies_df[['title', 'vote_average', 'genres', 'keywords']]

,title,vote_average,genres,keywords
2731,The Godfather: Part II,8.3,"[Drama, Crime]","[italo-american, cuba, vororte, melancholy, praise, revenge, mafia, lawyer, blood, corrupt politician, bloody body of child, man punches woman]"
1243,Mean Streets,7.2,"[Drama, Crime]","[epilepsy, protection money, secret love, money, redemption]"
3636,Light Sleeper,5.7,"[Drama, Crime]","[suicide, drug dealer, redemption, addict, existentialism]"
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0,"[Drama, Crime]","[police brutality, organized crime, policeman, illegal drugs, murder investigation, corrupt cop]"
2640,Things to Do in Denver When You're Dead,6.7,"[Drama, Crime]","[father son relationship, bounty hunter, boat, way of life, coffin, denver, godmother, paranoia, hitman, friendship, psychopath, revenge, murder, independent film, mafia, diner, blood, gangster, v..."
4065,Mi America,0.0,"[Drama, Crime]","[new york state, hate crime]"
1847,GoodFellas,8.2,"[Drama, Crime]","[prison, based on novel, florida, 1970s, mass murder, irish-american, drug traffic, biography, based on true story, murder, organized crime, gore, mafia, gangster, new york city, extreme violence,..."
4217,Kids,6.8,"[Drama, Crime]","[puberty, first time]"
883,Catch Me If You Can,7.7,"[Drama, Crime]","[con man, biography, fbi agent, overhead camera shot, attempted jailbreak, engagement party, mislaid trust, bank fraud, inspired by a true story]"
3866,City of God,8.1,"[Drama, Crime]","[male nudity, street gang, brazilian, photographer, 1970s, puberty, ghetto, gang war, coming of age, woman director, 1980s]"


- 대부 영화로 추천 필터링을 확인해보니 반드시 가장 유사한 영화가 해당 영화이지는 않다.
- 해당 영화를 제외한 영화를 추천하는 방식으로 추천 시스템을 만들어 볼 수 있다.

- 위 데이터프레임을 확인하니, 유사성이 높은 영화들을 추천하였으나 평점이 기록되지 않은 데이터가 확인된다.

In [22]:
# 영화에 대해 평점을 기준으로 정렬해볼 때 10개의 영화 확인
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False).head(10)

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


- vote_average가 높은 영화 중 vote_count가 확연히 낮은 데이터가 있는 것을 확인할 수 있다.
- 평점만 가지고 추천을 하기에는 위험하다는 판단할 수 있을 것 같다.
  - 이는 서비스 개발자의 판단에 따라 다르다.
- vote_average와 vote_count를 둘다 고려하는 방법으로 평점 가중치를 고려한 추천 필터링을 수정해보겠다.

### 평점 가중치 부여하기
투표 횟수가 적어도 vote_average가 높다면 평점이 높은 영화로 판단 될 수 있기 때문에 vote_count가 높은 상태에서도 높은 평점을 유지 하는 것이 정상적으로 평점이 높다고 할 수 있는 영화일 것이다.

새로운 가중평점은 다음과 같이 계산한다.
> - 아래 수식은 특정 논문에서 효율적인 가중평점을 구하는 식이다.

$$
가중평점(\text{Weighted Rating}) = R \times \frac{v}{(v+m)} + C \times \frac{m}{(v+m)}
$$
각 변수의 의미는 다음과 같다.
$$
v:\text{개별 영화에 평점을 투표한 횟수}\;\;m: \text{평점을 부여하기 위한 최소 투표 횟수}
$$
$$
R:\text{개별 영화에 대한 평균 평점}\;\;C: \text{전체 영화에 대한 평균 평점}
$$

예를 들어 A라는 영화의 투표 횟수가 1000회($v=1000$), 전체 데이터 세트 중에서 상위 60%의 투표 횟수가 300회($m=300$)라면 개별 영화 평점이 8.5점($R=8.5$), 전체 영화 평점이 6점($C=6.0$)일 때 다음과 같이 계산된다.
$$
8.5 \times \frac{1000}{1000 + 300} + 6.0 \times \frac{300}{1000+300}=7.92
$$


만약 상위 60%의 투표 횟수가 20회라면 어떻게 될까? 다른 영화들 보다 A라는 영화가 훨씬 더 투표에 많이 참여했기 때문에 더 많은 가중치를 받아 평점이 올라가게 된다.
$$
8.5 \times \frac{1000}{1000 + 20} + 6.0 \times \frac{20}{1000+20}=8.45
$$

In [23]:
# 전체 영화에 대한 평균 평점
C = movies_df['vote_average'].mean()

# 상위 60% 지점의 투표 횟수 구하기 - 임의로 지정했기 때문에 수정해도 괜찮다.
m = movies_df['vote_average'].quantile(0.6)

C, m

(6.092171559442016, 6.5)

In [25]:
## 개별 영화의 평점 가중치 구하는 함수 정의
def weighted_vote_average(record):
  # 개별영화의 투표 횟수와 평점 구하기
  v = record['vote_count']
  R = record['vote_average']

  # 가중 평점 공식 적용
  return (R * (v / (v + m))) + (C * (m / (v + m)))


## 개별 영화의 평점 가중치를 구해 DataFrame에 추가
movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1) # axis=0 열 벡터(Series), axis=1 행 벡터 이기 때문에 1로 넣어준다.

# 추가된 컬럼 확인
movies_df.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal,weighted_vote
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.",Action Adventure Fantasy Science Fiction,7.199390
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[ocean, drug abuse, exotic island, east india trading company, love of one's life, traitor, shipwreck, strong woman, ship, alliance, calypso, afterlife, fighter, pirate, swashbuckler, aftercredits...","Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.",Adventure Fantasy Action,6.898835
2,206647,Spectre,"[Action, Adventure, Crime]",6.3,4466,107.376788,"[spy, based on novel, secret agent, sequel, mi6, british secret service, united kingdom]","A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit ...",Action Adventure Crime,6.299698
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",7.6,9106,112.312950,"[dc comics, crime fighter, terrorist, secret identity, burglar, hostage drama, time bomb, gotham city, vigilante, cover-up, superhero, villainess, tragic hero, terrorism, destruction, catwoman, ca...","Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police...",Action Crime Drama Thriller,7.598924
4,49529,John Carter,"[Action, Adventure, Science Fiction]",6.1,2124,43.926995,"[based on novel, mars, medallion, space travel, princess, alien, steampunk, martian, escape, edgar rice burroughs, alien race, superhuman strength, mars civilization, sword and planet, 19th centur...","John Carter is a war-weary, former military captain who's inexplicably transported to the mysterious and exotic planet of Barsoom (Mars) and reluctantly becomes embroiled in an epic conflict. It's...",Action Adventure Science Fiction,6.099976


In [26]:
# 가중평점 기준 내림차순 정렬
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending=False).head(10)

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.498094,8205
3337,The Godfather,8.4,8.397457,5893
662,Fight Club,8.3,8.298476,9413
3232,Pulp Fiction,8.3,8.298299,8428
1818,Schindler's List,8.3,8.296690,4329
3865,Whiplash,8.3,8.296632,4254
2294,Spirited Away,8.3,8.296269,3840
2731,The Godfather: Part II,8.3,8.295709,3338
65,The Dark Knight,8.2,8.198859,12002
809,Forrest Gump,8.2,8.198273,7927


- 가중평점을 적용하기 전과 확연한 차이를 확인할 수 있다.

In [ ]:
## 가중 평점 기준으로 유사도 찾기 함수 재정의
def find_sim_movie(df, sorted_index, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values

    # top_n의 2배에 해당하는 장르 유사성이 높은 index 추출 - 유사도가 높은 조건과 함께 평점도 높여주기 위해 추천의 탐색 범위를 늘려준 것
    # 너무 값을 높일 경우 유사성이 떨어진 영화를 추천할 확률이 높아지므로 유의!
    similar_indexes = sorted_index[title_index, : top_n*2].reshape(-1)

    # 기준 영화는 제외하기(검색한 영화명에 대한 영화가 나오지 않도록)
    similar_indexes = similar_indexes[similar_indexes != title_index]

    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 정렬하여 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

In [27]:
## 가중평점 기준으로 'John Carter' 영화와 유사도가 높은 top 10 영화 확인
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'John Carter', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
102,The Hunger Games: Mockingjay - Part 2,6.6,6.599173
2995,Mad Max Beyond Thunderdome,5.9,5.901656
56,Star Trek Beyond,6.6,6.598718
85,Captain America: The Winter Soldier,7.6,7.598302
4117,Six-String Samurai,5.8,5.844685
47,Star Trek Into Darkness,7.4,7.398079
91,Independence Day: Resurgence,4.9,4.903103
207,Total Recall,7.1,7.096184
4042,U.F.O.,3.1,3.463535
2444,Damnation Alley,5.0,5.211914
